In [2]:
# Imports:
import numpy as np
import math
import torch
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from dataset import TreeClassifDataset, TreeClassifPreprocessedDataset

In [3]:
# Load and prepare data:

path = 'c:/Users/user/OneDrive/Dokumente/TUM/WS23/Data_Science_in_EO/DatSciEO/'

trees_dataset = TreeClassifPreprocessedDataset(f"{path}data/1123_delete_nan_samples_B2")
total_samples = len(trees_dataset)
input_shape = trees_dataset[0][0].shape

n_classes = trees_dataset[-1][1] + 1
n_channels = input_shape[0]
class_labels = [trees_dataset.label_to_labelname(trees_dataset[0][1]), trees_dataset.label_to_labelname(trees_dataset[480][1]), trees_dataset.label_to_labelname(trees_dataset[949][1]), trees_dataset.label_to_labelname(trees_dataset[1635][1]),
                trees_dataset.label_to_labelname(trees_dataset[3328][1]), trees_dataset.label_to_labelname(trees_dataset[3847][1]), trees_dataset.label_to_labelname(trees_dataset[6651][1]), trees_dataset.label_to_labelname(trees_dataset[9198][1]),
                trees_dataset.label_to_labelname(trees_dataset[10123][1]), trees_dataset.label_to_labelname(trees_dataset[10781][1])]

print(f"\nTotal length of dataset: {total_samples} \nShape of sample: {input_shape[1]}x{input_shape[2]} pixels over {n_channels} bands - {input_shape}")
print("Classes:", class_labels)
# print("\nClasses:")
# label = 200
# for i in range(total_samples):
    
#     if trees_dataset[i][1] != label:
#         label = trees_dataset[i][1]
#         print(label, trees_dataset.label_to_labelname(label))


BATCH_SIZE = 64

data_train, data_val = random_split(trees_dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(30))

train_dataloader = DataLoader(dataset=data_train, batch_size=BATCH_SIZE, shuffle=False)
val_dataloader = DataLoader(dataset=data_val, batch_size=BATCH_SIZE, shuffle=False)




Total length of dataset: 43944 
Shape of sample: 5x5 pixels over 29 bands - (29, 5, 5)
Classes: ['Acer_pseudoplatanus', 'Acer_pseudoplatanus', 'Acer_pseudoplatanus', 'Acer_pseudoplatanus', 'Betula_pendula', 'Carpinus_betulus', 'Fagus_sylvatica', 'Fagus_sylvatica', 'Fagus_sylvatica', 'Fagus_sylvatica']


In [4]:
# Define NN: taken from https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
# 2 conv layers, followed by 3 fully connected layers
# This is the network I used in the end

class NN_model(nn.Module):

    def __init__(self):
        super(NN_model, self).__init__()
        # 29 input image channels (bands), 2x2 square convolution
        # kernel
        self.conv1 = nn.Conv2d(n_channels, 35, 2, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(35, 50, 2, padding=1)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(50*2*2, 120)  
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, n_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Yealds ab 36% acc.
# With Adam: 45%
# data augmentation - 46%

In [ ]:
class dense_model(nn.Module):

    def __init__(self):
        super(dense_model, self).__init__()
        # 29 input image channels (bands), 2x2 square convolution
        # kernel
        self.fc1 = nn.Linear(n_channels*5*5, 145)
        self.fc2 = nn.Linear(145, 145)
        self.fc3 = nn.Linear(145, 29)
        self.fc4 = nn.Linear(29, 29)
        self.fc5 = nn.Linear(29, n_classes)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.softmax(self.fc5(x), dim=1)
        
        return x
    
# Yelds ab. 41% acc.

In [5]:
TreeClass_net = NN_model()

# Loss function & optimizer:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(TreeClass_net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(TreeClass_net.parameters())

In [5]:
# ResNet34: https://blog.paperspace.com/writing-resnet-from-scratch-in-pytorch/

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
    

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(29, 64, kernel_size = 3, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(5, stride=1)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        # x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
    

TreeClass_net = ResNet(ResidualBlock, [3, 4, 6, 3])

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(TreeClass_net.parameters(), lr=0.01, weight_decay = 0.001)  


In [6]:
# Train the Network:

N_EPOCHS = 50
n_iterations = math.ceil(total_samples/BATCH_SIZE)
print(total_samples, n_iterations)

for epoch in range(N_EPOCHS):
    
    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = TreeClass_net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print(f'[EPOCH {epoch + 1}] loss: {running_loss / n_iterations:.3f}')
    

print('Finished Training')


43944 687


In [9]:
# Save model:
torch.save(TreeClass_net.state_dict(), path+'TreeClassifier_Luca_cnn.pth')

In [7]:
# Test network on example validation data:
dataiter = iter(val_dataloader)
val_input, val_labels = next(dataiter)

outputs = TreeClass_net(val_input.float())
_, predicted = torch.max(outputs, 1)

print('Predictions: ', ' '.join(f'{predicted[j]}' for j in range(len(val_labels))))
print('Actual:', ' '.join(f'{val_labels[j]}' for j in range(len(val_labels))))

Predictions:  3 3 3 6 3 6 3 5 3 6 6 3 6 6 3 5 6 3 3 3 6 5 3 6 3 3 5 5 5 5 3 5
Actual: 4 8 8 5 3 5 5 5 3 6 6 7 6 6 3 5 5 2 3 3 6 0 4 6 7 0 6 6 6 5 7 6


In [8]:
# Test overall performance:
correct = 0
total = 0
with torch.no_grad():
    for data in val_dataloader:
        val_input, labels = data
        # calculate outputs by running images through the network
        outputs = TreeClass_net(val_input.float())
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(data_val)} test tree species: {100 * correct / total:3.3f} %')

Accuracy of the network on the 8788 test tree species: 46.040 %
